In [1]:
# automatic nested cross-validation of regression of 'Linear Regression using Step Forward Feature Selection' contrasted with 'Elastic Net' using pipelines as well as ZCA whitening to scale variables..

#https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/
#https://machinelearningmastery.com/elastic-net-regression-in-python/
#https://github.com/rasbt/mlxtend/issues/41
#https://github.com/rasbt/mlxtend/issues/69
#https://stackoverflow.com/questions/53252156/standardscaler-with-pipelines-and-gridsearchcv

In [2]:
from abc import ABC, abstractmethod

import itertools

import matplotlib.pyplot as plt

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from numpy import arange
from numpy import mean
from numpy import std
import numpy as np

import os

import pandas as pd

from random import sample

import random

import re

#from sklearn.datasets import make_classification
from sklearn import linear_model
from sklearn.datasets import make_regression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    warnings.filterwarnings('ignore')

In [3]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# ------------------------------------
# file: zca.py
# date: Thu May 21 15:47 2015
# author:
# Maarten Versteegh
# github.com/mwv
# maartenversteegh AT gmail DOT com
#
# Licensed under GPLv3
# ------------------------------------
"""zca: ZCA whitening with a sklearn-like interface

"""

from __future__ import division

import numpy as np
from scipy import linalg

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array, as_float_array

class ZCA(BaseEstimator, TransformerMixin):
    def __init__(self, regularization=1e-6, copy=False):
        self.regularization = regularization
        self.copy = copy

    def fit(self, X, y=None):
        """Compute the mean, whitening and dewhitening matrices.

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data used to compute the mean, whitening and dewhitening
            matrices.
        """
        X = check_array(X, accept_sparse=None, copy=self.copy,
                        ensure_2d=True)
        X = as_float_array(X, copy=self.copy)
        self.mean_ = X.mean(axis=0)
        X_ = X - self.mean_
        cov = np.dot(X_.T, X_) / (X_.shape[0]-1)
        U, S, _ = linalg.svd(cov)
        s = np.sqrt(S.clip(self.regularization))
        s_inv = np.diag(1./s)
        s = np.diag(s)
        self.whiten_ = np.dot(np.dot(U, s_inv), U.T)
        self.dewhiten_ = np.dot(np.dot(U, s), U.T)
        return self

    def transform(self, X, y=None, copy=None):
        """Perform ZCA whitening

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data to whiten along the features axis.
        """
        check_is_fitted(self, 'mean_')
        X = as_float_array(X, copy=self.copy)
        return np.dot(X - self.mean_, self.whiten_.T)

    def inverse_transform(self, X, copy=None):
        """Undo the ZCA transform and rotate back to the original
        representation

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data to rotate back.
        """
        check_is_fitted(self, 'mean_')
        X = as_float_array(X, copy=self.copy)
        return np.dot(X, self.dewhiten_) + self.mean_


In [4]:
def read_data():
    df = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv").set_index('States')
    return(df)

In [5]:
independent = 'Poverty'
outer_k = 10
inner_k = 10
random_st = random.sample(list(np.arange(0,10,1)),1)[0]
print(random_st)

# configure the cross-validation procedure
cv_inner = KFold(n_splits=inner_k, shuffle=True, random_state=random_st)
cv_outer = KFold(n_splits=outer_k, shuffle=True, random_state=random_st)

7


In [6]:
# create dataset
y = read_data()[[independent]]
X = read_data()[(read_data().columns).difference([independent]).values]

In [7]:
scores = list()

In [8]:
model_results = list()

In [94]:
cv = KFold(n_splits=10, random_state=1,shuffle=True)
ratios = arange(0, 1, 0.1)
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
model = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, n_jobs=-1)

outer_results = list()

# perform cross-validation procedure
for train_ix, test_ix in cv_outer.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]

    
    pipe_ENet_Regressor = Pipeline([
                #('scaler',  ZCA()),
                ('ENet_Regressor', model)])

    grid_params_ENet_Regressor = [{
    }]


    CV_ENet_regressor = GridSearchCV (estimator = pipe_ENet_Regressor,
                                   param_grid = grid_params_ENet_Regressor,
                                   #cv = cv,return_train_score=True, verbose=0, refit=True)
                                   return_train_score=True, verbose=0, refit=True)

    CV_ENet_regressor.fit(X_train, y_train)
    
    print(best_model.named_steps['ENet_Regressor'].coef_)
    
    #ypred=CV_ENet_regressor.predict(X_test)

    best_model = CV_ENet_regressor.best_estimator_
    yhat = best_model.predict(X_test)

    # evaluate the model
    acc = mean_squared_error(y_test, yhat,squared=True)
    #cross_val_score(self.search, X, y, scoring='neg_mean_squared_error', cv=cv_outer, n_jobs=-1)
    # store the result
    outer_results.append(acc)
    
print('accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

model_results.append(['Elastic',mean(outer_results), std(outer_results)])

[ 2.11959562e-03  4.48151309e-03 -3.11204546e-04  2.38830513e-03
  7.44896599e-08  6.29854278e-04  4.50184327e-03 -1.62534937e-03
 -3.62924054e-02]
[-1.89849098e-03  3.84660107e-03 -2.73498343e-04  1.77463610e-01
  6.77979869e-08  2.01048769e+00  5.74691096e-01  7.19957977e-02
 -4.79904187e-02]
[-1.05293748e-03  1.54512845e-03 -2.84741120e-04 -3.97770678e-01
  3.85962180e-08  4.15823982e-01  3.47371244e-01 -1.00772561e-01
 -1.38011535e-01]
[ 3.97825928e-04  3.66528333e-03 -3.32627568e-04  0.00000000e+00
  8.17126604e-08  0.00000000e+00  1.59696119e-01 -0.00000000e+00
 -6.29880719e-02]
[ 2.03819990e-03  5.77465111e-03 -3.34100289e-04  1.99041139e-03
  7.99484943e-08  8.22960475e-04  4.32081178e-03 -3.03706008e-03
 -3.43229831e-02]
[ 1.34172989e-03  2.67440757e-03 -3.19084379e-04  0.00000000e+00
  8.32421881e-08  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -5.42062977e-02]
[ 1.53981622e-03  3.20085343e-03 -3.39259667e-04  0.00000000e+00
  8.85301308e-08  0.00000000e+00  0.00000000e+0

In [91]:
#linear

cv = KFold(n_splits=2, random_state=1,shuffle=True)
#model = ElasticNetCV(l1_ratio=0, alphas=[0], n_jobs=-1)
model = LinearRegression()

outer_results = list()

class sfs_(SFS):        
    def score(self):
        return np.min(np.abs([self.subsets_[i]['avg_score'] for i in range(1,len(self.subsets_))]))*-1

# perform cross-validation procedure
for train_ix, test_ix in cv_outer.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]

    sfs1 = sfs_(model, 
               k_features=len(X.columns),
               forward=True, 
               floating=False, 
               scoring='neg_mean_squared_error',
               cv=cv)

    pipe = Pipeline([
                    #('standardize', ZCA()),
                     ('sfs', sfs1)#, 
                     #('lr', model)
                    ])

    param_grid = {
      }

    CV_Linear_regressor = GridSearchCV (estimator = pipe,
                                   param_grid = param_grid,
                                   #cv = cv,return_train_score=True, verbose=0, refit=True)
                                   verbose=0, refit=True)

    CV_Linear_regressor.fit(X_train, y_train)
    
    #ypred=CV_ENet_regressor.predict(X_test)

    best_model = CV_Linear_regressor.best_estimator_
    #plt.plot(pd.DataFrame(best_model.named_steps['sfs'].subsets_).loc['avg_score'])

    bestSFS=X.columns[pd.Series(pd.DataFrame(best_model.named_steps['sfs'].subsets_).iloc[:,np.argmin(np.abs(pd.DataFrame(best_model.named_steps['sfs'].subsets_).loc['avg_score']))-1].feature_idx).to_list()]
    print(bestSFS)
    lm = LinearRegression()
    lm.fit(X_train[bestSFS],y_train)
    
    #print(best_model.named_steps['lr'].coef_)
    yhat = lm.predict(X_test[bestSFS])

    # evaluate the model
    acc = mean_squared_error(y_test, yhat,squared=True)
    #cross_val_score(self.search, X, y, scoring='neg_mean_squared_error', cv=cv_outer, n_jobs=-1)
    # store the result
    outer_results.append(acc)
    
print('accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

model_results.append(['Linear',mean(outer_results), std(outer_results)])

Index(['Income', 'Infant Mort', 'Population', 'Unemployed'], dtype='object')
Index(['Income', 'Infant Mort', 'Population', 'Unemployed', 'University',
       'White'],
      dtype='object')
Index(['Income', 'Population', 'Unemployed', 'University', 'White'], dtype='object')
Index(['Income', 'Population', 'Unemployed', 'White'], dtype='object')
Index(['Income', 'Population', 'White'], dtype='object')
Index(['Doctors', 'Income', 'Population', 'Traf Deaths', 'Unemployed'], dtype='object')
Index(['Doctors', 'Income', 'Population', 'Traf Deaths', 'Unemployed',
       'White'],
      dtype='object')
Index(['Crime', 'Income', 'Population', 'Traf Deaths', 'Unemployed'], dtype='object')
Index(['Doctors', 'Income', 'Population', 'Traf Deaths', 'White'], dtype='object')
Index(['Doctors', 'Income', 'Population', 'Traf Deaths', 'Unemployed',
       'University', 'White'],
      dtype='object')
accuracy: 2.791 (2.876)


In [51]:
X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]

sfs1 = SFS(model, 
           k_features=len(X.columns),
           forward=True, 
           floating=False, 
           scoring='neg_mean_squared_error',
           cv=cv)
sfs1.fit(X,y)


SequentialFeatureSelector(cv=KFold(n_splits=2, random_state=1, shuffle=True),
                          estimator=LinearRegression(), k_features=9,
                          scoring='neg_mean_squared_error')

In [88]:
dir(sfs_)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_calc_confidence',
 '_check_fitted',
 '_check_n_features',
 '_exclusion',
 '_get_param_names',
 '_get_params',
 '_get_tags',
 '_inclusion',
 '_more_tags',
 '_replace_estimator',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_set_params',
 '_validate_data',
 '_validate_names',
 'fit',
 'fit_transform',
 'get_metric_dict',
 'get_params',
 'named_estimators',
 'score',
 'set_params',
 'transform']

AttributeError: 'SequentialFeatureSelector' object has no attribute 'subsets'

-1.7864075324941493

In [42]:
sfs1.subsets_

{}

In [11]:
model_results

[['Elastic', 2.6994513749989006, 2.9898170221851905],
 ['Linear', 2.7511100931572363, 3.0819880480803907]]

In [13]:
for s in range(0,len(model_results)):
    print('Model: ', model_results[s][0])
    print('Accuracy: %.3f (%.3f)' % (model_results[s][1], model_results[s][2]))

#step 1
best_model = [item[0] for item in model_results][np.argmin(np.abs([item[1] for item in model_results]))]
print(best_model)

Model:  Elastic
Accuracy: 2.277 (1.963)
Model:  Linear
Accuracy: 2.385 (1.936)
Elastic


In [13]:
"""
class model(ABC):
 
    @abstractmethod
    def setEstimators(self):
        estimators_ENetCV = []
        estimators_ENetCV.append(('standardize', ZCA()))

        pass
"""

class regression_model():
    def __init__(self,estimators):
        self.estimators = estimators
        
        # define search
        self.search = Pipeline(self.estimators)
        # execute the nested cross-validation
        self.scores = cross_val_score(self.search, X, y, scoring='neg_mean_squared_error', cv=cv_outer, n_jobs=-1)
        # report performance
        self.score = [(mean(self.scores), std(self.scores))]
        
    def score(self):
        
        return(self.score)

In [ ]:
#elastic

# define the model

estimators_ENetCV = []
estimators_ENetCV.append(('standardize', ZCA()))
model = ElasticNet()
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
#grid['n_alphas'] = [10]
grid['l1_ratio'] = arange(0, 1, 0.1)
search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', n_jobs=1, cv=cv_inner, refit=True)
#estimators_ENetCV.append(('ElasticNetCV', ElasticNetCV(cv=cv_inner, n_alphas=100, random_state=random_st,fit_intercept=1)))
estimators_ENetCV.append(('ElasticNet', search))

elastic_score = regression_model(estimators_ENetCV)
scores.append(['ElasticNet',elastic_score])

In [ ]:
#search.get_params().keys()

In [ ]:
search.fit(X,y)

In [ ]:
search

In [ ]:
search.best_estimator_

In [ ]:
#linear

# define the model

#linear_enet = ElasticNetCV(cv=cv_inner,alphas=[0], l1_ratio=0,fit_intercept = True)
linear_enet = ElasticNetCV(cv=cv_inner,alphas=[0], l1_ratio=0,fit_intercept = True)

sfs = SFS(linear_enet, 
          k_features=len(X.columns), 
          forward=True, 
          floating=True, 
          scoring='neg_mean_squared_error',
          cv=cv_inner)

estimators_linear = []
estimators_linear.append(('standardize', ZCA()))
estimators_linear.append(('SFS', sfs))
estimators_linear.append(('linear_enet', linear_enet))
grid = dict()
grid['alpha'] = [0]
grid['l1_ratio'] = [0]
search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', n_jobs=1, cv=cv_inner, refit=True)

linear_score = regression_model(estimators_linear)
scores.append(['linear_enet',linear_score])

In [ ]:
for s in range(0,len(scores)):
    print('Model: ', scores[s][0])
    print('Accuracy: %.3f (%.3f)' % (scores[s][1].score[0][0], scores[s][1].score[0][1]))


In [ ]:
#step 1
best_model = [item[0] for item in scores][np.argmin(np.abs([item[1].score[0][0] for item in scores]))]
print(best_model)

In [ ]:


if(best_model=='ElasticNetCV'):
    zca = ZCA()
    zca.fit(X)
    
    #Step 2: inner-procedure is applied to the entire dataset.
    final_model_est = Pipeline(estimators_ENetCV)
    final_model_est.fit(X,y)
    #final_model_est = ElasticNetCV(alphas=[final_model_est.named_steps['ElasticNetCV'].alpha_], l1_ratio=final_model_est.named_steps['ElasticNetCV'].l1_ratio,fit_intercept=True)
    
    #final_model_est.fit(zca.transform(X), y)
    
    #step 3: hyperparameters found during this final search are then used to configure a final model.
    estimators_final = []
    #estimators_final.append(('standardize', ZCA()))
    estimators_final.append(('ElasticNet', ElasticNetCV(cv=cv_inner,alphas=[final_model_est.named_steps['ElasticNetCV'].alpha_], l1_ratio=final_model_est.named_steps['ElasticNetCV'].l1_ratio,fit_intercept=True)))
    final_model = Pipeline(estimators_final)
    
    final_model.fit(zca.transform(X),y)    
    
    #step 4: final model is fit on the entire dataset.
    plt.scatter(final_model.predict(zca.transform(X)),y)
    plt.show()
    print(mean_squared_error(y, final_model.predict(zca.transform(X)), squared=True))
    
    #Step 2: inner-procedure is applied to the entire dataset.
    final_model_est = Pipeline(estimators_ENetCV)
    final_model_est.fit(X,y)
    #final_model_est = ElasticNetCV(alphas=[final_model_est.named_steps['ElasticNetCV'].alpha_], l1_ratio=final_model_est.named_steps['ElasticNetCV'].l1_ratio,fit_intercept=True)
    
    #final_model_est.fit(zca.transform(X), y)
    
    #step 3: hyperparameters found during this final search are then used to configure a final model.
    estimators_final = []
    #estimators_final.append(('standardize', ZCA()))
    estimators_final.append(('ElasticNet', ElasticNetCV(cv=cv_inner,alphas=[final_model_est.named_steps['ElasticNetCV'].alpha_], l1_ratio=final_model_est.named_steps['ElasticNetCV'].l1_ratio,fit_intercept=True)))
    final_model = Pipeline(estimators_final)
    
    final_model.fit(final_model_est.named_steps['standardize'].transform(X),y)    
    
    #step 4: final model is fit on the entire dataset.
    plt.scatter(final_model.predict(final_model_est.named_steps['standardize'].transform(X)),y)
    plt.show()
    print(mean_squared_error(y, final_model.predict(final_model_est.named_steps['standardize'].transform(X)), squared=True))
        
elif(best_model=='linear_enet'):
    final_model_est = Pipeline(estimators_linear)
    
    final_model_est.fit(X,y)
    
    bestSFS=X.columns[pd.Series(pd.DataFrame(final_model_est.named_steps['SFS'].subsets_).iloc[:,np.argmin(np.abs(pd.DataFrame(final_model_est.named_steps['SFS'].subsets_).loc['avg_score']))-1].feature_idx).to_list()]
    
    #Step 2: inner-procedure is applied to the entire dataset.
    #step 3: hyperparameters found during this final search are then used to configure a final model.
    #no need to derive hyper parm's
    final_model = ElasticNetCV(cv=cv_inner,alphas=[0], l1_ratio=0,fit_intercept=True)
    
    zca = ZCA()
    zca.fit(X[bestSFS])
    
    final_model.fit(zca.transform(X[bestSFS]), y)
    
    #final_model.fit(zca.transform(X),y)    
    
    #step 4: final model is fit on the entire dataset.
    plt.scatter(final_model.predict(zca.transform(X[bestSFS])),y)
    plt.show()
    print(mean_squared_error(y, final_model.predict(zca.transform(X[bestSFS])), squared=True))
